## HOMEWORK 1

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

### Read the data for January. How many columns are there?

In [3]:
data_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
data_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
print(len(data_jan.columns))
print(len(data_feb.columns))

19
19


### What's the standard deviation of the trips duration in January?

In [4]:
data_jan['duration'] = data_jan['tpep_dropoff_datetime'] - data_jan['tpep_pickup_datetime']
data_jan['duration_minutes'] = data_jan['duration'].dt.total_seconds() / 60
print(data_jan['duration_minutes'].std())

42.594351241920904


In [5]:
data_feb['duration'] = data_feb['tpep_dropoff_datetime'] - data_feb['tpep_pickup_datetime']
data_feb['duration_minutes'] = data_feb['duration'].dt.total_seconds() / 60
print(data_feb['duration_minutes'].std())

42.84210176105097


### What fraction of the records left after you dropped the outliers?

In [6]:
train_data_clean = data_jan[data_jan['duration_minutes'].between(1.0, 60.0)]
difference = len(data_jan['duration_minutes']) - len(train_data_clean['duration_minutes'])
print(f"We removed {(100*difference/len(data_jan['duration_minutes'])):0.2f} % of the rows.")
print(f"So there is {100 - 1.88}% left.")

We removed 1.88 % of the rows.
So there is 98.12% left.


In [7]:
val_data_clean = data_feb[data_feb['duration_minutes'].between(1.0, 60.0)]
difference = len(data_feb['duration_minutes']) - len(val_data_clean['duration_minutes'])
print(f"We removed {(100*difference/len(data_feb['duration_minutes'])):0.2f} % of the rows.")
print(f"So there is {100 - 1.99}% left.")

We removed 1.99 % of the rows.
So there is 98.01% left.


## Linear regression 


In [8]:
train_location_data = pd.DataFrame({
    'PULocationID': train_data_clean['PULocationID'].apply(str),
    'DOLocationID': train_data_clean['DOLocationID'].apply(str)
})
train_location_dicts = train_location_data.to_dict('records')

# For validation data
val_location_data = pd.DataFrame({
    'PULocationID': val_data_clean['PULocationID'].apply(str),
    'DOLocationID': val_data_clean['DOLocationID'].apply(str)
})
val_location_dicts = val_location_data.to_dict('records')


: 

### Apply one hot encoding to PULocation and to DOLocation
One-hot encoding is a technique used to convert categorical data into a binary format where each category is represented by a separate column with a 1 indicating its presence and 0s for all other categories.

In [8]:
vectorizer = DictVectorizer(sparse=True)
X_train = vectorizer.fit_transform(train_location_dicts)
print(f"Shape of training matrix: {X_train.shape}")
print(f"Number of features: {len(vectorizer.get_feature_names_out())}")

y_train = train_data_clean['duration_minutes'].values
print(len(y_train))

Shape of training matrix: (3009173, 515)
Number of features: 515
3009173


In [9]:
X_val = vectorizer.transform(val_location_dicts)
print(f"Shape of val matrix: {X_val.shape}")
print(f"Number of features: {len(vectorizer.get_feature_names_out())}")

y_val = val_data_clean['duration_minutes'].values
print(f"Len val {len(y_val)}")

Shape of val matrix: (2855951, 515)
Number of features: 515
Len val 2855951


In [10]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [12]:
train_pred = model.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
print(f"RMSE on training data: {train_rmse:.4f} minutes")

RMSE on training data: 7.6493 minutes


In [13]:
val_pred = model.predict(X_val)
val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
print(f"RMSE on val data: {val_rmse:.4f} minutes")

RMSE on val data: 7.8118 minutes
